In [92]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [109]:
# df = pd.read_csv('clean_games.csv')
# df.to_json('full_json.json')

In [110]:
df = pd.read_json('full_json.json')
df

,Unnamed: 0,Title,Release Date,Genre,Platforms,Developer,ESRB Rating,ESRBs,Meta Crtic score,User Score,Meta Critic Reviews,user_reviews,Number of Players,Game Summary
0,0,Burnout 3: Takedown,"Sep 7, 2004","Driving, Racing, Arcade",Xbox,Criterion Games,T,Mild Language Mild Violence,94,7.7,76,191.0,"1-2 Players, 8 Players Online",Burnout 3 challenges you to crash into (and th...
1,1,Jet Grind Radio,"Oct 30, 2000","Action, Platformer, 3D",Dreamcast,Smilebit,T,Animated Violence Mild Language,94,8.3,24,105.0,1 Player,"Join a graffiti crew, stamp your territory and..."
2,2,Metal Gear Solid 4: Guns of the Patriots,"Jun 12, 2008","Action Adventure, Modern, General, Modern",PlayStation 3,Kojima Productions,M,Blood Crude Humor Strong Language Suggestive T...,94,8.8,82,4231.0,"1 Player, 16 Players Online Up to 16 Players",Metal Gear Solid 4: Guns of the Patriots featu...
3,3,Tom Clancy's Splinter Cell Chaos Theory,"Mar 28, 2005","Action Adventure, Modern",Xbox,Ubisoft Montreal,M,Blood Strong Language Violence,94,9.1,70,233.0,"1-4 Players, 4 Players Online","As Sam Fisher, Third Echelon's most skilled Sp..."
4,4,Call of Duty: Modern Warfare 2,"Nov 10, 2009","Action, Shooter, Shooter, First-Person, Modern...",Xbox 360,Infinity Ward,M,Blood Drug Reference Intense Violence Language,94,6.6,100,3055.0,Up to 18 Players,Modern Warfare 2 continues the gripping and he...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19312,19312,Family Party: 30 Great Games Obstacle Arcade,"Dec 4, 2012","Miscellaneous, Party, Party, Party / Minigame",Wii U,"Art Co., Ltd.",E,Mild Cartoon Violence,11,2.5,8,179.0,1 Player,Family Party: 30 Great Games Obstacle Arcade b...
19313,19313,Ride to Hell: Retribution,"Jun 24, 2013","Driving, Modern, Racing, Motorcycle, Motocross...",PC,"Eutechnyx, Deep Silver",M,Blood and Gore Drug Reference Intense Violence...,16,1.4,9,603.0,1 Player,The game is set in the last years of the roari...
19314,19314,Far Cry 5: Lost on Mars,"Jul 17, 2018","Action, Shooter, First-Person, Arcade",PC,Ubisoft Montreal,M,None,61,4.5,12,53.0,1 Player,"Nick Rye, the ""King of the Skies"" is ready to ..."
19315,19315,Smash T.V.,"Nov 24, 2005","Action, Shooter, Shooter, Static, Static, Shoo...",Xbox 360,Digital Eclipse,T,Blood and Gore Violence,75,8.3,15,14.0,"1-2 Players, 2 Players 2 Players Online",[Xbox Live Arcade] The future of reality TV i...


In [96]:
liked_games = df.sample(n=10)
likes = []
for i in range(10):
    likes.append(1)
liked_games['like_status'] = likes

In [97]:
non_liked_games = df.sample(n=20)
non_likes = []
for i in range(20):
    non_likes.append(0)
non_liked_games['like_status'] = non_likes

In [98]:
train_df = pd.concat([non_liked_games,liked_games], ignore_index=True)
train_df.head(5)

,Unnamed: 0,title,release_date,genre,platforms,developer,esrb_rating,ESRBs,metascore,userscore,critic_reviews,user_reviews,num_players,summary,like_status
0,13002,World Championship Poker,"Nov 30, 2004","Miscellaneous, Parlor, Gambling",Xbox,Coresoft,None,None,66,5.6,7,5.0,1 Player,Enter the fast and freewheeling world of high-...,0
1,411,Silent Hill 2,"Sep 24, 2001","Action Adventure, Horror",PlayStation 2,KCET,M,Blood and Gore Violence,89,9.2,34,1246.0,1 Player,"Players assume the role of James, who receives...",0
2,8933,Spellcaster University,"Sep 24, 2019","Strategy, Adventure, General, Management, Busi...",PC,Sneaky Yak Studio,None,None,73,8.0,7,6.0,1 Player,Develop a prestigious university of mages. Bui...,0
3,10274,Lake,"Sep 1, 2021","Adventure, General",Xbox Series X,Gamious,M,None,71,6.6,15,20.0,1 Player,It's 1986 - Meredith Weiss takes a break from...,0
4,14228,Silent Hill: Downpour,"Mar 13, 2012","Action Adventure, Horror, Horror, Survival",PlayStation 3,Vatra,M,Blood and Gore Intense Violence Sexual Themes ...,64,6.9,41,322.0,1 Player,The survival horror experience begins after a ...,0


In [99]:
train_df.drop(['Unnamed: 0','summary'], axis=1,inplace=True)
train_df.head()

,title,release_date,genre,platforms,developer,esrb_rating,ESRBs,metascore,userscore,critic_reviews,user_reviews,num_players,like_status
0,World Championship Poker,"Nov 30, 2004","Miscellaneous, Parlor, Gambling",Xbox,Coresoft,None,None,66,5.6,7,5.0,1 Player,0
1,Silent Hill 2,"Sep 24, 2001","Action Adventure, Horror",PlayStation 2,KCET,M,Blood and Gore Violence,89,9.2,34,1246.0,1 Player,0
2,Spellcaster University,"Sep 24, 2019","Strategy, Adventure, General, Management, Busi...",PC,Sneaky Yak Studio,None,None,73,8.0,7,6.0,1 Player,0
3,Lake,"Sep 1, 2021","Adventure, General",Xbox Series X,Gamious,M,None,71,6.6,15,20.0,1 Player,0
4,Silent Hill: Downpour,"Mar 13, 2012","Action Adventure, Horror, Horror, Survival",PlayStation 3,Vatra,M,Blood and Gore Intense Violence Sexual Themes ...,64,6.9,41,322.0,1 Player,0


In [100]:
train_df.drop_duplicates(inplace=True)

In [101]:
dummy_df = pd.get_dummies(train_df)
the_input_dim = dummy_df.shape[1] - 1

In [102]:
X = dummy_df.drop('like_status',axis=1)
y = dummy_df['like_status']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [103]:
scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [104]:
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=60, activation='relu', input_dim=the_input_dim))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 60)                10860     
                                                                 
 dense_22 (Dense)            (None, 30)                1830      
                                                                 
 dense_23 (Dense)            (None, 1)                 31        
                                                                 
Total params: 12,721
Trainable params: 12,721
Non-trainable params: 0
_________________________________________________________________


In [105]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [106]:
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
1/1 [==============================] - 1s 728ms/step - loss: nan - accuracy: 0.7273
Epoch 2/100
1/1 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.6818
Epoch 3/100
1/1 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.6818
Epoch 4/100
1/1 [==============================] - 0s 8ms/step - loss: nan - accuracy: 0.6818
Epoch 5/100
1/1 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.6818
Epoch 6/100
1/1 [==============================] - 0s 15ms/step - loss: nan - accuracy: 0.6818
Epoch 7/100
1/1 [==============================] - 0s 16ms/step - loss: nan - accuracy: 0.6818
Epoch 8/100
1/1 [==============================] - 0s 12ms/step - loss: nan - accuracy: 0.6818
Epoch 9/100
1/1 [==============================] - 0s 12ms/step - loss: nan - accuracy: 0.6818
Epoch 10/100
1/1 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.6818
Epoch 11/100
1/1 [==============================] - 0

In [107]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1/1 - 0s - loss: nan - accuracy: 0.6250 - 302ms/epoch - 302ms/step
Loss: nan, Accuracy: 0.625
